# Exercise 10: 

In this week's exercise, we will be working from the class demo. To use the live features, you'll need to get a Twitter account approved through the Developer Portal (Links to an external site.) as reviewed in class. If you can't get approval, you can use the alternate method of importing your data via TAGS (Links to an external site.)

Your application should:

- Import or collect tweets using a keyword or hashtag. Watch your rate limiting!
- Use Pandas to sort and analyze your tweets.
- Calculate user coordinates, where feasible, for the set of collected tweets. Again, watch your rate limiting!
- Map the tweets using the calculated locations.
- Use at least one data visualization from a previous exercise, demo, or the course text to analyze a different element of your tweets.

As a bonus challenge, try to map some other quality of your data. For instance, you might only map the locations where a certain word or hashtag is used, or try to visualize the quantity of tweets that share a location using a larger radius dot.


## Stage One: Import or collect tweets using a keyword or hashtag. Watch your rate limiting!


## Stage Two:  Use Pandas to sort and analyze your tweets.

## Stage Three: Calculate user coordinates, where feasible, for the set of collected tweets. Again, watch your rate limiting! 

## Stage Four:  Map the tweets using the calculated locations.

## Stage Five: Use at least one data visualization from a previous exercise, demo, or the course text to analyze a different element of your tweets.

## Bonus Challenge: Try to map some other quality of your data. For instance, you might only map the locations where a certain word or hashtag is used, or try to visualize the quantity of tweets that share a location using a larger radius dot.

